In [46]:
!pip install psycopg2

   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   --------- ------------------------------ 0.3/1.2 MB ? eta -:--:--
   ------------------ --------------------- 0.5/1.2 MB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 2.2 MB/s eta 0:00:00


In [47]:
import psycopg2
from psycopg2 import sql
import pandas as pd

In [3]:
df = pd.read_csv('C://Users//PC//Documents//Chapa//Data//transaction.csv')
df.head()

,id,mobile,amount,payment_type,ref_id,status,attempts,device,created_at,updated_at,completed_at
0,c4ac2d32-d1f1-4d29-bc18-d8c436134966,932065985,11601.87,chapa_bank_7,ft_JP4N46EPIIKJ,1,1,android,2025-05-31 14:34:07.417448,2025-05-31 14:34:47.417448,2025-05-31 14:34:47.417448
1,29644ae7-b34c-4133-816f-534e969d7597,997822985,7836.47,chapa_bank_10,ft_WPMBMS23NZK7,1,1,web,2025-07-14 14:17:34.637397,2025-07-14 14:17:59.637397,2025-07-14 14:17:59.637397
2,584e6fb3-dcf1-4378-8b80-25becbac0c6a,978764787,8026.88,chapa_bank_8,ft_PT2CCKG1I5VW,1,1,android,2025-06-21 04:21:26.893276,2025-06-21 04:21:34.893276,2025-06-21 04:21:34.893276
3,4c1e1d3c-2991-4488-81b5-8663cf76fab1,970388081,4350.67,chapa_bank_8,ft_AE5720JQQ234,1,1,android,2025-06-04 06:00:18.083075,2025-06-04 06:01:04.083075,2025-06-04 06:01:04.083075
4,c5dc93e5-48b6-4e66-8849-d66821a5dd70,939399629,12287.39,chapa_bank_3,ft_O81S8PN21RRR,1,1,android,2025-07-12 19:02:01.315022,2025-07-12 19:02:50.315022,2025-07-12 19:02:50.315022


In [ ]:
df.isnull().sum()

id                 0
mobile             0
amount             0
payment_type       0
ref_id             0
status             0
attempts           0
device             0
created_at         0
updated_at         0
completed_at    8097
dtype: int64

- there is no completed_at time for 8097 transactions beacuse this transactions was not successfull (failed)

In [45]:
# number of successful vs missing completed_at
print("Successful transactions with completed_at:", df[df['status'] == 1]['completed_at'].notnull().sum())
print("Non-successful transactions with completed_at:", df[df['status'] != 1]['completed_at'].notnull().sum())


Successful transactions with completed_at: 71903
Non-successful transactions with completed_at: 0


In [6]:
df['created_at'] = pd.to_datetime(df['created_at'], errors='coerce')
df['updated_at'] = pd.to_datetime(df['updated_at'], errors='coerce')
df['completed_at'] = pd.to_datetime(df['completed_at'], errors='coerce')

### time to complete (for successful ones)

In [7]:
df['time_to_complete'] = (df['completed_at'] - df['created_at']).dt.total_seconds()

In [10]:
df[:3]

,id,mobile,amount,payment_type,ref_id,status,attempts,device,created_at,updated_at,completed_at,time_to_complete
0,c4ac2d32-d1f1-4d29-bc18-d8c436134966,932065985,11601.87,chapa_bank_7,ft_JP4N46EPIIKJ,1,1,android,2025-05-31 14:34:07.417448,2025-05-31 14:34:47.417448,2025-05-31 14:34:47.417448,40.0
1,29644ae7-b34c-4133-816f-534e969d7597,997822985,7836.47,chapa_bank_10,ft_WPMBMS23NZK7,1,1,web,2025-07-14 14:17:34.637397,2025-07-14 14:17:59.637397,2025-07-14 14:17:59.637397,25.0
2,584e6fb3-dcf1-4378-8b80-25becbac0c6a,978764787,8026.88,chapa_bank_8,ft_PT2CCKG1I5VW,1,1,android,2025-06-21 04:21:26.893276,2025-06-21 04:21:34.893276,2025-06-21 04:21:34.893276,8.0


### friction flag

In [12]:
df['is_friction'] = (df['status'] != 1) | (df['attempts'] > 1)
df[:5]

,id,mobile,amount,payment_type,ref_id,status,attempts,device,created_at,updated_at,completed_at,time_to_complete,is_friction
0,c4ac2d32-d1f1-4d29-bc18-d8c436134966,932065985,11601.87,chapa_bank_7,ft_JP4N46EPIIKJ,1,1,android,2025-05-31 14:34:07.417448,2025-05-31 14:34:47.417448,2025-05-31 14:34:47.417448,40.0,False
1,29644ae7-b34c-4133-816f-534e969d7597,997822985,7836.47,chapa_bank_10,ft_WPMBMS23NZK7,1,1,web,2025-07-14 14:17:34.637397,2025-07-14 14:17:59.637397,2025-07-14 14:17:59.637397,25.0,False
2,584e6fb3-dcf1-4378-8b80-25becbac0c6a,978764787,8026.88,chapa_bank_8,ft_PT2CCKG1I5VW,1,1,android,2025-06-21 04:21:26.893276,2025-06-21 04:21:34.893276,2025-06-21 04:21:34.893276,8.0,False
3,4c1e1d3c-2991-4488-81b5-8663cf76fab1,970388081,4350.67,chapa_bank_8,ft_AE5720JQQ234,1,1,android,2025-06-04 06:00:18.083075,2025-06-04 06:01:04.083075,2025-06-04 06:01:04.083075,46.0,False
4,c5dc93e5-48b6-4e66-8849-d66821a5dd70,939399629,12287.39,chapa_bank_3,ft_O81S8PN21RRR,1,1,android,2025-07-12 19:02:01.315022,2025-07-12 19:02:50.315022,2025-07-12 19:02:50.315022,49.0,False


In [ ]:
status_change = df
status_change['completed_at'] = status_change['completed_at'].apply(lambda x: None if pd.isna(x) else x)

#### data inserted to db 

In [79]:
# Database connection parameters
db_host = '127.0.0.1'
db_name = 'chapa_transaction'
db_user = 'postgres'
db_password = '*************'

# Load your DataFrame (example if not already loaded)
# status_change = pd.read_csv("transactions.csv")

# Convert string timestamps to datetime
status_change['created_at'] = pd.to_datetime(status_change['created_at'], errors='coerce')
status_change['updated_at'] = pd.to_datetime(status_change['updated_at'], errors='coerce')
status_change['completed_at'] = pd.to_datetime(status_change['completed_at'], errors='coerce')

# Add status_label
status_change['status_label'] = status_change['status'].map({0: 'Pending', 1: 'Success', 2: 'Failed'})

# Add friction flag
status_change['is_friction'] = (status_change['status'] != 1) | (status_change['attempts'] > 1)

# Add time_to_complete (only for successful transactions)
status_change['time_to_complete'] = (status_change['completed_at'] - status_change['created_at']).dt.total_seconds()
status_change['time_to_complete'] = status_change.apply(
    lambda x: x['time_to_complete'] if x['status'] == 1 else None, axis=1
)

# Confirm nulls in critical fields
print("Null check:")
print(status_change[['created_at', 'updated_at', 'completed_at', 'time_to_complete']].isnull().sum())

try:
    # Connect to the PostgreSQL database
    connection = psycopg2.connect(
        host=db_host,
        database=db_name,
        user=db_user,
        password=db_password
    )

    cursor = connection.cursor()

    # SQL query to insert data
    insert_query = sql.SQL("""
        INSERT INTO payment.transaction (
            mobile, amount, payment_type, ref_id, status, attempts,
            device, created_at, updated_at, completed_at,
            time_to_complete, is_friction, status_label
        )
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """)

    # Insert each record
    for index, record in status_change.iterrows():
        # Convert pandas Timestamps to Python datetimes
        created_at = record['created_at'].to_pydatetime() if pd.notnull(record['created_at']) else None
        updated_at = record['updated_at'].to_pydatetime() if pd.notnull(record['updated_at']) else None
        completed_at = record['completed_at'].to_pydatetime() if pd.notnull(record['completed_at']) else None
        time_to_complete = float(record['time_to_complete']) if pd.notnull(record['time_to_complete']) else None

        cursor.execute(insert_query, (
            record['mobile'],
            record['amount'],
            record['payment_type'],
            record['ref_id'],
            record['status'],
            record['attempts'],
            record['device'],
            created_at,
            updated_at,
            completed_at,
            time_to_complete,
            record['is_friction'],
            record['status_label']
        ))

        # print(f"Inserted record {index}")

    # Commit the transaction
    connection.commit()
    print("✅ Data inserted successfully.")

except Exception as e:
    print(f"❌ An error occurred: {e}")

finally:
    # Close the cursor and connection
    if 'cursor' in locals():
        cursor.close()
    if 'connection' in locals():
        connection.close()



Null check:
created_at             0
updated_at             0
completed_at        8097
time_to_complete    8097
dtype: int64
✅ Data inserted successfully.


In [21]:
status_change['status_label'] = status_change['status'].map({0: 'Pending', 1: 'Success', 2: 'Failed'})
status_change[:4]

,id,mobile,amount,payment_type,ref_id,status,attempts,device,created_at,updated_at,completed_at,time_to_complete,is_friction,status_label
0,c4ac2d32-d1f1-4d29-bc18-d8c436134966,932065985,11601.87,chapa_bank_7,ft_JP4N46EPIIKJ,1,1,android,2025-05-31 14:34:07.417448,2025-05-31 14:34:47.417448,2025-05-31 14:34:47.417448,40.0,False,Success
1,29644ae7-b34c-4133-816f-534e969d7597,997822985,7836.47,chapa_bank_10,ft_WPMBMS23NZK7,1,1,web,2025-07-14 14:17:34.637397,2025-07-14 14:17:59.637397,2025-07-14 14:17:59.637397,25.0,False,Success
2,584e6fb3-dcf1-4378-8b80-25becbac0c6a,978764787,8026.88,chapa_bank_8,ft_PT2CCKG1I5VW,1,1,android,2025-06-21 04:21:26.893276,2025-06-21 04:21:34.893276,2025-06-21 04:21:34.893276,8.0,False,Success
3,4c1e1d3c-2991-4488-81b5-8663cf76fab1,970388081,4350.67,chapa_bank_8,ft_AE5720JQQ234,1,1,android,2025-06-04 06:00:18.083075,2025-06-04 06:01:04.083075,2025-06-04 06:01:04.083075,46.0,False,Success


In [80]:
status_change['amount'].sum()

603508498.6700001

In [23]:
print("📦 Basic Stats")
print(f"Total Transactions: {len(status_change)}")
print(f"Successful: {len(status_change[status_change.status == 1])}")
print(f"Pending: {len(status_change[status_change.status == 0])}")
print(f"Failed: {len(status_change[status_change.status == 2])}")
print(f"Friction Transactions: {status_change['is_friction'].sum()}")

print("\n💰 Amount Stats")
print(f"Min Amount: {status_change.amount.min():,.2f} Birr")
print(f"Max Amount: {status_change.amount.max():,.2f} Birr")
print(f"Average Amount: {status_change.amount.mean():,.2f} Birr")


📦 Basic Stats
Total Transactions: 80000
Successful: 71903
Pending: 4001
Failed: 4096
Friction Transactions: 11636

💰 Amount Stats
Min Amount: 50.26 Birr
Max Amount: 14,999.92 Birr
Average Amount: 7,543.86 Birr


### Success Rate by Payment Type

In [24]:
success_by_payment = status_change.groupby('payment_type')['status'].apply(lambda x: (x == 1).mean() * 100).sort_values(ascending=False)
print("\n✅ Success Rate by Payment Type:\n", success_by_payment)



✅ Success Rate by Payment Type:
 payment_type
chapa_bank_5     90.348723
chapa_bank_3     90.117380
chapa_bank_10    90.104232
chapa_bank_6     90.007628
chapa_bank_1     89.950311
chapa_bank_2     89.908601
chapa_bank_4     89.791328
chapa_bank_8     89.580172
chapa_bank_9     89.501604
chapa_bank_7     89.478261
Name: status, dtype: float64


### Avg Amount by Device

In [30]:
avg_amount_by_device = status_change.groupby('device')['amount'].mean()
print("\n📱 average transaction Amount in Birr by Device:\n", avg_amount_by_device)



📱 average transaction Amount in Birr by Device:
 device
android    7534.662526
ios        7551.346973
web        7547.257465
Name: amount, dtype: float64


### Time to Completion (Success Only)

In [31]:
completion_stats = status_change[status_change.status == 1]['time_to_complete'].describe()
print("\n⏳ Time to Completion (Success Only):\n", completion_stats)



⏳ Time to Completion (Success Only):
 count    71903.000000
mean        32.402236
std         16.175580
min          5.000000
25%         18.000000
50%         32.000000
75%         46.000000
max         60.000000
Name: time_to_complete, dtype: float64


- max time to complete a transaction has taken 60 seconds and the minimum is 5 seconds.
- the average time to complete 71903 transaction successfully is 32 seconds.


#### Attempts

In [32]:
multi_attempts = status_change[status_change.attempts > 1]
print(f"\n🔁 Transactions with Multiple Attempts: {len(multi_attempts)}")
print(f"Percentage: {(len(multi_attempts) / len(df)) * 100:.2f}%")



🔁 Transactions with Multiple Attempts: 3539
Percentage: 4.42%


In [34]:
multi_attempts = status_change[status_change.attempts > 1]
multi_attempts['status_label'].value_counts()

status_label
Success    3539
Name: count, dtype: int64

- from the whole transaction completed by customers 4.42% of transaction was completed with attempts

### transaction status starting may 15, 2025 to jul 16, 2025

In [35]:
status_change['date'] = status_change['created_at'].dt.date
transactions_per_day = status_change.groupby('date')['id'].count()
success_per_day = status_change[status_change.status == 1].groupby('date')['id'].count()
failed_per_day = status_change[status_change.status == 2].groupby('date')['id'].count()

# Merge into single DataFrame for export or visualization
trend_df = pd.DataFrame({
    'total': transactions_per_day,
    'success': success_per_day,
    'failed': failed_per_day
}).fillna(0)


In [ ]:
transactions_per_days = trend_df.reset_index()
transactions_per_days

,date,total,success,failed
0,2025-05-16,638,565,37
1,2025-05-17,1316,1186,64
2,2025-05-18,1297,1166,63
3,2025-05-19,1385,1231,77
4,2025-05-20,1406,1278,69
...,...,...,...,...
56,2025-07-11,1307,1190,59
57,2025-07-12,1306,1171,77
58,2025-07-13,1335,1198,70
59,2025-07-14,1398,1254,69


### Friction by Device and Payment Type

In [43]:
friction_by_device = status_change[status_change.is_friction].groupby('device')['id'].count()
friction_by_payment = status_change[status_change.is_friction].groupby('payment_type')['id'].count()

print("\n❌ Friction by Device:\n", friction_by_device)
print("\n❌ Friction by Payment Type:\n", friction_by_payment)



❌ Friction by Device:
 device
android    4706
ios        4623
web        2307
Name: id, dtype: int64

❌ Friction by Payment Type:
 payment_type
chapa_bank_1     1150
chapa_bank_10    1131
chapa_bank_2     1170
chapa_bank_3     1163
chapa_bank_4     1168
chapa_bank_5     1145
chapa_bank_6     1131
chapa_bank_7     1168
chapa_bank_8     1201
chapa_bank_9     1209
Name: id, dtype: int64


### Conclusion
The analysis presented in this transaction report provides valuable insights into the performance and efficiency of payment processing across various devices and payment types. Key findings include:

- Friction Rate Analysis: The friction rate varies significantly by device, indicating that user experience can be enhanced on specific platforms to reduce transaction failures.

- Average Time to Complete Transactions: The average completion time for transactions shows discrepancies among different payment types, suggesting areas for optimization in processing speed.

- Revenue Loss Identification: The report highlights substantial revenue losses due to failed and pending transactions, emphasizing the need for strategies to minimize these occurrences.

- Failed Transactions by Device: Specific devices report higher rates of transaction failures, warranting further investigation into their compatibility and performance.

- Multi-Attempt Patterns: The analysis of multi-attempt counts reveals user behavior trends that can inform improvements in the payment process.

- Transaction Amount Variability: The distribution of transaction amounts per device offers insights into revenue generation potential and user spending behavior.

- Status Label Insights: Understanding the status labels assigned to transactions helps in assessing overall transaction health and identifying areas for intervention.

By addressing the identified issues and leveraging the insights gained, the company can enhance its payment processing system, improve user satisfaction, and ultimately drive higher revenue. Ongoing monitoring and analysis will be essential to ensure sustained improvements and adapt to evolving user needs.

